# Dot Size
This notebook is used to determine the appropriate size of the quantum dot in a dot superconductor dot system

In [2]:
try:
    import nqcpfem
except ModuleNotFoundError as err:
    import sys
        # put path to nqcpfem module here   
    src_path = '/mnt/c/Users/swao/nqcp/NQCP_FEM/'
    sys.path.append(src_path)
    import nqcpfem
import sympy
sympy.init_printing(use_latex='mathjax')
import IPython.display as disp
from matplotlib import pyplot as plt
import numpy as np
import os

import logging
LOG = logging.getLogger()
LOG.setLevel(logging.INFO)
from nqcpfem.parameter_search import DBMPParameterSearch, ParameterSearch
from nqcpfem.band_model import LuttingerKohnHamiltonian
from nqcpfem.solvers import PETScSolver
from nqcpfem.envelope_function import RectangleDomain
from nqcpfem.fenics import FEniCsModel
from nqcpfem.g_matrix import GMatrix
from nqcpfem.band_model import __MAGNETIC_FIELD_NAMES__ as _B_
_B_ = sympy.symbols(_B_,commutative=False) # cast to symbols
from nqcpfem.functions import SymbolicFunction





# Single setup Test

In [2]:
bm = LuttingerKohnHamiltonian(spatial_dim=3).material_spec('Ge')
bm.add_z_confinement(1,'box',25e-9)
#bm.add_zeeman_term()#B=[1,1,1])



solver = PETScSolver(k=10,which='SM',sigma=0)
domain = RectangleDomain(Lx=250e-9,Ly=150e-9,Lz=0) 
domain.resolution = [300,180]
fem_model = FEniCsModel(bm,domain,0,('CG',1))


x,y,z = bm.position_symbols
omega_x,omega_y = sympy.symbols(r'\omega_x,\omega_y')
m = sympy.symbols('m')

V = 0.5*m*( (omega_x*x)**2 + (omega_y*y)**2 ) 

bm.add_potential(V)

bm.material_spec('Ge')

E0 = 1/(nqcpfem.UNIT_CONVENTION['J to eV']*1000000) # unit_scale: 1muev
omega_y_val = 2e12
omega_x_val = omega_y_val


fem_model.band_model.parameter_dict[omega_y] = omega_y_val

fem_model.band_model.parameter_dict[omega_x] = omega_x_val


In [3]:
sol = solver.solve(fem_model)

(217924, 217924)


In [4]:
sol[0]/E0

array([ 8230.79545378+0.j,  8230.79545378+0.j, 12473.19768838+0.j,
       12473.19768838+0.j, 13117.10694604+0.j, 13117.10694604+0.j,
       14454.48202257+0.j, 14454.48202257+0.j, 15818.35334776+0.j,
       15818.35334776+0.j, 16872.97825275+0.j])

In [5]:
import pyvista
from nqcpfem.plotting import plot_eigenvector

In [36]:
logging.disabled = True
plot_eigenvector(sol[1][7],fem_model)

INFO:trame_server.controller:trigger(trigger__29)
INFO:trame_server.controller:trigger(trigger__30)
INFO:trame_server.controller:trigger(P_0x7f079a519010_4Camera)
INFO:trame_server.controller:trigger(P_0x7f079a519010_4AnimateStart)
INFO:trame_server.controller:trigger(P_0x7f079a519010_4AnimateStop)
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:js_key = fluid
INFO:trame_client.widgets.core:js_key = class
INFO:trame_client.widgets.core:before: class = { 'rounded-circle': !P_0x7f079a519010_4_show_ui }
INFO:trame_server.utils.namespace:(prefix=None) token {
INFO:trame_server.state:has({ => {) = False
INFO:trame_server.utils.namespace:(prefix=None) token  
INFO:trame_server.state:has(  =>  ) = False
INFO:trame_server.utils.namespace:(prefix=None) token '
INFO:trame_server.state:has(' => ') = False
INFO:trame_server.utils.namespace:(prefix=None) token rounded
INFO:trame_server.state:has(rounded => rounded) = False
INFO:trame_server.utils.namespace:(prefix=None)

Widget(value='<iframe src="http://localhost:40061/index.html?ui=P_0x7f079a519010_4&reconnect=auto" class="pyvi…

In [9]:
from nqcpfem.observables import Observable

In [23]:
from nqcpfem.band_model import __POSITION_NAMES__
import sympy
X = sympy.symbols(__POSITION_NAMES__[0],commutative=False)
Xop = sympy.Array([[X,0,0,0],[0,X,0,0],[0,0,X,0],[0,0,0,X]])
Xsqop = sympy.Array([[X**2,0,0,0],[0,X**2,0,0],[0,0,X**2,0],[0,0,0,X**2]])
from nqcpfem.fenics import FEniCsObservable
fenics_x_op = FEniCsObservable(Xop,fem_model)
fenics_xsq_op = FEniCsObservable(Xsqop,fem_model)

In [30]:
for evec in sol[1]:
    print(fenics_x_op.mel(evec)*1e9)


-2.1949393183475348e-14
7.187441250748525e-14
-1.928509123296552e-10
9.314223527765688e-14
4.34363668896213e-11
-1.9990751791180278e-13
-2.339391924667222e-12
-4.9115688744341466e-11
-7.976886135559109e-08
-1.2912522370868735e-09
3.4991850924250834e-11


In [34]:
for evec in sol[1]:
    print(np.sqrt(fenics_xsq_op.mel(evec))*1e9)

10.605674371734414
10.605674371734356
13.046937909471158
13.046937909470042
14.381946173052972
14.381946173068322
11.222530431636258
11.222530431637729
14.409578922343066
14.409578958884497
13.24331553530087


# Size Parameter Search

## Model Initial setup

In [3]:
bm = LuttingerKohnHamiltonian(spatial_dim=3).material_spec('Ge')
bm.add_z_confinement(1,'box',25e-9)
#bm.add_zeeman_term()#B=[1,1,1])



solver = PETScSolver(k=10,which='SM',sigma=0)
domain = RectangleDomain(Lx=300e-9,Ly=300e-9,Lz=0) 
domain.resolution = [300,300]
fem_model = FEniCsModel(bm,domain,0,('CG',1))


x,y,z = bm.position_symbols
omega_x,omega_y = sympy.symbols(r'\omega_x,\omega_y')
m = sympy.symbols('m')

V = 0.5*m*( (omega_x*x)**2 + (omega_y*y)**2 ) 

bm.add_potential(V)

bm.material_spec('Ge')

E0 = 1/(nqcpfem.UNIT_CONVENTION['J to eV']*1000000) # unit_scale: 1muev
omega_y_val = 2e12
omega_x_val = omega_y_val


fem_model.band_model.parameter_dict[omega_y] = omega_y_val

fem_model.band_model.parameter_dict[omega_x] = omega_x_val


from nqcpfem.band_model import __POSITION_NAMES__
import sympy
X = sympy.symbols(__POSITION_NAMES__[0],commutative=False)
Xop = sympy.Array([[X,0,0,0],[0,X,0,0],[0,0,X,0],[0,0,0,X]])
Xsqop = sympy.Array([[X**2,0,0,0],[0,X**2,0,0],[0,0,X**2,0],[0,0,0,X**2]])
from nqcpfem.fenics import FEniCsObservable
fenics_x_op = FEniCsObservable(Xop,fem_model)
fenics_xsq_op = FEniCsObservable(Xsqop,fem_model)





## Parameter Search definition

In [4]:
from nqcpfem.parameter_search import DBParameterSearch


omega_range = np.linspace(1e11,2e12,128)




def size_meaurement(omega):
    fem_model.band_model.parameter_dict[omega_x] =omega
    fem_model.band_model.parameter_dict[omega_y] =omega

    # solve the system
    solution = solver.solve(fem_model)

    # extract only the numbers that we care about
    sq_vals = []
    last_E = -np.inf
    for eval,evec in zip(*solution):
        if not np.isclose(eval,last_E,rtol=1e-3,atol=0):
            # compute expectation value of X^2
            sq_vals.append(np.sqrt(fenics_xsq_op.mel(evec))*1e9)
            last_E = eval/E0
        else:
            pass
    print(sq_vals)
    return sq_vals
            

search = DBParameterSearch(omega_range,size_meaurement,'size_meaurement.save')




In [5]:
search.run(True,False)

INFO:nqcpfem.parameter_search:running database parametersearch with 128 points
INFO:nqcpfem.parameter_search:evaluating at grid point 1/128


(362404, 362404)


INFO:nqcpfem.parameter_search:evaluating at grid point 2/128


[45.74309720676824, 45.743097194581736, 60.980417044923016, 60.980417044993935, 60.888852610269666, 60.88885261031149, 72.9463985342915, 72.94639852276485, 66.799531319719, 66.79953132407584, 67.05104407958618]
(362404, 362404)


INFO:nqcpfem.parameter_search:evaluating at grid point 3/128


...

[36.86205921191531, 36.86205921171254, 51.31792579895623, 51.31792579878691, 51.32995193276769, 51.32995193251938, 62.293754839241856, 62.29375483926847, 60.19147908725349, 60.191479087140095, 60.50007169342029, 60.500071689503216, 69.74675024171152, 69.64620793987822]
(362404, 362404)


INFO:nqcpfem.parameter_search:evaluating at grid point 8/128


[35.45233461380159, 35.45233461368427, 49.534040232609065, 49.53404023356563, 49.56049302406373, 49.56049302420199, 60.17429365233183, 60.174293651374114, 58.63336540469655, 58.63336540461035, 58.95380665953905, 58.95380662877936, 67.74638010406551, 67.67254192967079]
(362404, 362404)


INFO:nqcpfem.parameter_search:evaluating at grid point 9/128


[34.16816865303161, 34.16816865296964, 47.85389063597231, 47.853890635667256, 47.89259873195038, 47.89259873178908, 58.1496326517872, 58.149632651437834, 57.054813029179755, 57.05481302890532, 57.389658461884395, 57.3896584538751, 65.73230152357812, 65.66414091218341]


## Magnetic Field Sweep

In [ ]:
OMEGA_Y_VAL = 0


B_field_magnitudes = np.linspace(0.1,2,16)
Lx_values =  np.linsapce(0.5,2,64)*OMEGA_Y_VAL
param_dict = [{'b':b,'lx':lx} for b in B_field_magnitudes for lx in Lx_values]

from nqcpfem.spin_gap import find_spin_gap

def sho_eval_b_field(lx,b):
    #make band_model and fem model
    bm = LuttingerKohnHamiltonian(spatial_dim=3).material_spec('Ge')
    bm.add_z_confinement(1,'box',25e-9)
    bm.add_zeeman_term()#B=[1,1,1])



    solver = PETScSolver(k=10,which='SM',sigma=0)
    domain = RectangleDomain(Lx=250e-9,Ly=150e-9,Lz=0) 
    domain.resolution = [300,180]
    fem_model = FEniCsModel(bm,domain,0,('CG',1))


    x,y,z = bm.position_symbols
    omega_x,omega_y = sympy.symbols(r'\omega_x,\omega_y')
    m = sympy.symbols('m')

    V = 0.5*m*( (omega_x*x)**2 + (omega_y*y)**2 ) 

    bm.add_potential(V)

    bm.material_spec('Ge')

    E0 = 1/(nqcpfem.UNIT_CONVENTION['J to eV']*1000000) # unit_scale: 1muev
    omega_y_val = 2e12


    fem_model.band_model.parameter_dict[omega_y] = omega_y_val

    fem_model.band_model.parameter_dict[omega_x] = 1e12 #omega_x_val


    sho_gmat = GMatrix(fem_model,solver)
    sho_gmat.envelope_model.band_model.parameter_dict[omega_y]


    for bname in _B_:
    
        bm.function_dict[bname] = SymbolicFunction(sympy.sympify(0),bname)
    
    fem_model = sho_gmat.envelope_model
    fem_model.band_model.parameter_dict[omega_x] = lx#omega_x_val
    fem_model.band_model.function_dict[_B_[0]]= SymbolicFunction(sympy.sympify(b),_B_[0])
    fem_model.band_model.function_dict[_B_[1]]= SymbolicFunction(sympy.sympify(0),_B_[1])
    #compute Bx energies
    
    
    Bx_energies = find_spin_gap(solver.solve(fem_model),fem_model)
    LOG.debug(Bx_energies)

    
    #compute By energies
    fem_model.band_model.function_dict[_B_[0]]= SymbolicFunction(sympy.sympify(0),_B_[0])
    fem_model.band_model.function_dict[_B_[1]]= SymbolicFunction(sympy.sympify(b),_B_[1])
    By_energies = find_spin_gap(solver.solve(fem_model),fem_model)
    LOG.debug(By_energies)
    
    fem_model.band_model.function_dict[_B_[1]]= SymbolicFunction(sympy.sympify(0),_B_[1])
    
    del fem_model
    
    return Bx_energies,By_energies
    
from nqcpfem.parameter_search import ParameterSearch,DBMPParameterSearch
sho_save = 'sho_fem_b_field.save'

#Lx_values = np.linspace(50e-9,200e-9,64)

sho_search  = None 

if os.path.exists(sho_save):
    try:
        sho_search = ParameterSearch.load(sho_save)
    except Exception as err:
        print(err)
#sho_search=None
#sho_search = None
if sho_search is None:
    sho_search = DBMPParameterSearch(parameter_sets = param_dict,evaluation_function=sho_eval_b_field,save_file=sho_save)